In [ ]:
# install
! pip3 install --user --upgrade kfp==1.8.13 --quiet

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

In [ ]:
#####################################################################
#
# setup
#
#####################################################################

In [ ]:
# imports for this notebook to run
from google.cloud import aiplatform as vertex

import kfp
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, component)

In [ ]:
# specify parameters
PROJECT_ID = "your-project-id"
REGION = "us-central1"
BUCKET_NAME = f"bkt-{PROJECT_ID}-vpipelines"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_PATH}/pipeline_root"
PIPELINE_DATA = f"{BUCKET_PATH}/data"

In [ ]:
#####################################################################
# BEGIN vertex pipelines
#####################################################################

In [ ]:
#####################################################################
#
# create individual pipeline components, then specify the pipeline
#
#####################################################################

In [ ]:
# create a pipeline component
@component(base_image="python:3.9", output_component_file="step_1.yaml")
def step_1():
    raise Exception("This task is broken!")

In [ ]:
# define a pipeline
@dsl.pipeline(name="my-pipeline-name", description="my pipeline description")
def my_pipeline(project_id: str = PROJECT_ID):    
    # specify the nodes in the pipeline
    #step_1_task = step_1()
    step_1_task = (step_1().set_retry(num_retries=10, backoff_duration="60s", backoff_factor=2.0, backoff_max_duration="1000s"))

In [ ]:
#####################################################################
#
# compile and run the pipeline
#
#####################################################################

In [ ]:
# compile the pipeline
my_package_path = 'my_vertex_pipeline_specification_file.json'
compiler.Compiler().compile(pipeline_func=my_pipeline, package_path=my_package_path)

In [ ]:
# run the pipeline
vertex.init(project=PROJECT_ID)

job = vertex.PipelineJob(
    display_name = "my-pipeline-job-name",
    template_path = my_package_path,
    pipeline_root = PIPELINE_ROOT,
    enable_caching = False
)

job.submit()